In [3]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import math


### Set Initial Parameters. Here, the control group has a mean outcome of 40 while the treatment has a mean outcome of 60. They have equal variance. The formula calculates the total sample size necessary to detect this difference with 80% power

In [4]:
np.random.seed(12345)
iterations = 1000

control_mean = 40
treatment_mean = 50
control_std = 70
treatment_std = 70
N = math.ceil(2*(control_std**2+treatment_std**2)*((2.8/(treatment_mean-control_mean))**2))
print(N)

1537


### Create a data set of length N as required to detect the difference in means above. Split the sample equally into treatment and control. Control and treatment obs draw from the distributions as defined above.

In [6]:
x = np.random.normal(loc=control_mean,scale=control_std,size=N)
n2 = round(N/2)
df = pd.DataFrame(np.random.uniform(0,1,size=(N,3)), columns=list(['treatment','control_draw','treated_draw']))
df.iloc[0:n2,0]=0
df.iloc[n2:,0]=1
df['control_draw'] = np.random.normal(loc=control_mean,scale=control_std,size=N)
df['treated_draw'] = np.random.normal(loc=treatment_mean,scale=treatment_std,size=N)
df['outcome'] = np.where(df['treatment']==0,df['control_draw'],df['treated_draw'])
print(df.groupby('treatment')['outcome'].agg('mean'))

treatment
0.0    40.536164
1.0    48.089782
Name: outcome, dtype: float64


In [3]:
def create_sample(N,control_mean,control_std,treatment_mean,treatment_std):
  n2 = round(N/2)
  df = pd.DataFrame(np.random.uniform(0,1,size=(N,3)), columns=list(['treatment','control_draw','treated_draw']))
  df.iloc[0:n2,0]=0
  df.iloc[n2:,0]=1
  df['control_draw'] = np.random.normal(loc=control_mean,scale=control_std,size=N)
  df['treated_draw'] = np.random.normal(loc=treatment_mean,scale=treatment_std,size=N)
  df['outcome'] = np.where(df['treatment']==0,df['control_draw'],df['treated_draw'])
  return df

### For each iteration, we generate a new sample and estimate a linear regression that is equivalent to a two-sample t-test. Save the t-statistics from each simulation.

In [4]:
def get_t_list(iterations,N):
  tlist = list()
  for i in range(iterations):
    df = create_sample(N,control_success,treatment_success)
    formula = 'outcome~treatment'
    model = smf.ols(data=df,formula=formula).fit()
    t=model.tvalues[1]
    tlist.append(t)
  return tlist

### Show one example simulation

In [5]:
df = create_sample(N,control_success,treatment_success)
formula = 'outcome~treatment'
model = smf.ols(data=df,formula=formula).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.706
Date:                Mon, 31 May 2021   Prob (F-statistic):            0.00966
Time:                        13:56:04   Log-Likelihood:                -1817.4
No. Observations:                2745   AIC:                             3639.
Df Residuals:                    2743   BIC:                             3651.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3054      0.013     24.104      0.0

### With the appropriate sample size and sufficiently large number of iterations, the share of simulations that are statistically significant should be very close to 80% by design. 

In [6]:
tlist = get_t_list(iterations,N)
df1 = pd.DataFrame(tlist,columns=['tstat'])
df1['sig'] = np.where(np.abs(df1['tstat'])>=1.96,1,0)
print(df1['sig'].mean())

0.806
